Из этого всего  вам не нужны только последние четыре импорта(пока что)

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Лютые функции, которые вы так ждёте

необязательна, просто взял её из ручной работы. Может пригодиться

In [2]:
def iqr_filter(df: pd.DataFrame, column: str, lower_bound=True, upper_bound=True, multp=3):
  """
  гибкая функция для удаления выбросов с помощью настраиваемого интерквартильного размаха

  df: DataFrame содержащий данные, которые нужно отфильтровать.

  column: Название столбца в DataFrame df, в которомом будет производиться фильтрация.
  Функция будет рассчитывать IQR именно для этого столбца.

  lower_bound:  Булевый флаг, определяющий, следует ли отфильтровывать значения,
  лежащие ниже нижней границы, рассчитанной на основе IQR. То есть, если
  lower_bound = False, то все выбросы(если они есть) будут игнорироваться.

  upper_bound: Булевый флаг, определяющий, следует ли отфильтровывать значения,
  лежащие выше верхней границы, рассчитанной на основе IQR.

  multp: Множитель, используемый для расчета границ фильтрации.
  """
  q1, q3 = np.percentile(df[column], [25, 75])
  iqr = (q3 - q1) * multp
  low_bound = q1 - iqr
  up_bound = q3 + iqr

  median = 0 # медиана
  outlines = 0 # индексы, значения строк столбца которых необходимо заменить медианой

  # в этих ситуациях мы ищем медиану чисел, которые не считаем за выбросы (1)
  # то есть мы считаем за выбросы up_bound или/и low_bound и не используем их диапазон значений
  # для поиска медианы.
  if lower_bound and upper_bound:
    outlines = df[(df[column] < low_bound) | (df[column] > up_bound)].index
    median = df[(df[column] >= low_bound) & (df[column] <= up_bound)][column].median() # та самая медиана (1)
  elif lower_bound:
    outlines = df[df[column] < low_bound].index
    median = df[df[column] >= low_bound][column].median() # та самая медиана (1)
  elif upper_bound:
    outlines = df[df[column] > up_bound].index
    median = df[df[column] <= up_bound][column].median() # та самая медиана (1)

  df.loc[outlines, column] = median

####################################################################

Постараюсь ввести в курс дела:
**ОБЯЗАТЕЛЬНАЯ ФУНКЦИЯ ДЛЯ ЗАМЕНЫ NaNов**

df - датафрейм со всеми столбцами кроме того, в котором мы заполняем пропуски

y - столбец с пропусками, которые необходимо заполнить

КАК РАБОТАЕТ

начнём с цикла:

У нас есть проблема - для заполенния пропусков с помощью какой-либо модели, необходимо, чтобы ВСЕ значения в других признаках были заполнены(не было пропусков). В противном случае ругается, что есть NaNы.

Данный цикл устраняет данную проблему, временно заполняя пропуски в столбцах на медиану всех значений признака(кроме столбца, задача для которого изначально была заполнить пропуски с помощью модели).

Дальше смотрите по комментариям

In [30]:
def super_train_split(df: pd.DataFrame, y):
  X = df.copy()
  for col in X.columns:
    if X[col].isnull().any() == True:
      median_value = X[col].median()
      X[col].fillna(median_value, inplace=True)


  y_train = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
  y_temp = y[y.isnull()] # просто мусор. Полезный

  idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
  X_train = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

  idxs = y_train.index # берём иднексы c изначально заполенными значениями
  X_test = X.drop(idxs) # отбрасываем строки с заполненными значениями в нужном нам столбце.
                        # Получается выборка с данными, на основе которых будут
                        # предсказываться пропущенные значения

  return X_train, X_test, y_train, y_temp

##########################################################

Данная функция для вас не обязательна. Нужна была для тестов.

In [29]:
def split_for_grade(df: pd.DataFrame, target_column: str):
  df = df.dropna(subset=[target_column]).copy()

  y = df[target_column]
  df = df.drop('stage_4_output_danger_gas', axis=1)

  for col in df.columns:
    if df[col].isnull().any() == True:
      median_value = df[col].median()
      df[col].fillna(median_value, inplace=True)

  X = df.drop(target_column, axis=1)
  X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42)

  return X_train, X_test, y_train, y_test

# Нужная рутина

инициализация моделей. **ВАМ НУЖНА ТОЛЬКО ПЕРВАЯ СТРОЧКА**

In [6]:
random_forest_model = RandomForestRegressor(random_state=42)
linear_regression = LinearRegression()
knn_model = KNeighborsRegressor(n_neighbors=3)

In [7]:
df_environmental_data = pd.read_csv("analysing_environmental_issues.csv", sep=',') # главный датафрейм

In [8]:
df_environmental_data

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_danger_gas,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
0,2021-01-28 18:00:00,67.83,92.99,474.18,76.84,97.52,49.94,361.50,252.04,97.48,...,108.65,45.59,156.67,19.08,5.92,356.05,NaN,21.48,47.03,2.0
1,2021-01-28 19:00:00,67.83,91.82,473.68,76.15,97.82,48.55,354.75,244.87,97.66,...,108.71,45.89,156.76,19.15,5.94,357.69,NaN,21.48,45.05,2.0
2,2021-01-28 20:00:00,67.83,90.65,473.17,75.46,98.12,47.15,348.00,237.70,97.85,...,108.76,46.19,156.86,19.23,5.97,359.33,NaN,21.48,43.06,2.0
3,2021-01-28 21:00:00,67.93,90.24,473.59,75.26,97.79,49.33,356.74,249.87,97.50,...,108.63,45.87,156.41,19.36,5.97,339.99,0.16,21.48,46.01,2.0
4,2021-01-28 22:00:00,68.03,89.84,474.00,75.06,97.46,51.51,365.49,262.04,97.15,...,108.51,45.54,155.96,19.49,5.97,320.64,NaN,21.48,48.95,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,2023-03-19 12:00:00,67.33,43.32,409.32,82.00,105.65,69.35,771.29,223.71,97.37,...,111.26,41.04,152.90,22.69,4.14,381.37,0.20,21.18,9.17,2.0
4396,2023-03-20 00:00:00,68.83,62.24,377.84,79.82,104.80,61.08,781.76,183.36,97.70,...,110.09,46.40,152.53,21.38,4.15,381.77,0.14,21.08,10.79,2.0
4397,2023-03-20 12:00:00,66.83,61.92,397.95,81.19,105.50,66.26,776.52,227.64,98.31,...,108.65,45.99,152.53,21.78,4.14,391.60,0.20,21.08,9.27,1.0
4398,2023-03-21 01:00:00,70.03,62.80,378.37,79.16,110.50,63.16,845.66,223.15,103.01,...,108.18,47.70,151.80,22.44,3.66,460.04,0.18,22.38,10.11,1.0


столбец DateTime типа object. Для корректной работы его лучше перевести в datetime типа

In [ ]:
df_environmental_data['DateTime'] = pd.to_datetime(df_environmental_data['DateTime'])

Просто копия главного датафрейма для экспериментов

In [31]:
df = df_environmental_data.copy()

Регрессионным моделям не нравится столбец DateTime, поэтому я его пока что просто удалил, но можно поставить в качестве индекса. DateTime сам по себе не очень полезен(на этапе предобработки).   

In [32]:
df.pop('DateTime')

,DateTime
0,2021-01-28 18:00:00
1,2021-01-28 19:00:00
2,2021-01-28 20:00:00
3,2021-01-28 21:00:00
4,2021-01-28 22:00:00
...,...
4395,2023-03-19 12:00:00
4396,2023-03-20 00:00:00
4397,2023-03-20 12:00:00
4398,2023-03-21 01:00:00


# Проверка разных моделей (Вам это не нужно копировать). Можете просто игнорировать этот раздел

так как мы должны пояснять все свои действия, я сделал заготовку ответа на вопрос "почему именно такой метод для замены NaNов"

**RandomForestRegressor**

In [ ]:
X_train, X_test, y_train, y_test = split_for_grade(df, 'stage_1_output_konv_avd')

random_forest_model.fit(X_train, y_train)

y_pred = random_forest_model.predict(X_test)

print("r2: ", r2_score(y_test, y_pred),               # чем больше число, тем лучше. От 0 до 1
      "\nMSE: ", mean_squared_error(y_test, y_pred))  # чем меньше, тем лучше. от 0 до бесконечности

**LinearRegression**

In [ ]:
X_train, X_test, y_train, y_test = split_for_grade(df, 'stage_1_output_konv_avd')

linear_regression.fit(X_train, y_train)

y_pred = linear_regression.predict(X_test)

print("r2: ", r2_score(y_test, y_pred),              # чем больше число, тем лучше. От 0 до 1
      "\nMSE: ", mean_squared_error(y_test, y_pred)) # чем меньше, тем лучше. от 0 до бесконечности

r2:  0.47668198346620416 
MSE:  7.6313619061749245


<ipython-input-229-6d0e2b802b15>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_value, inplace=True)


**KNeighborsRegressor**

In [ ]:
X_train, X_test, y_train, y_test = split_for_grade(df, 'stage_1_output_konv_avd')

knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)

print("r2: ", r2_score(y_test, y_pred),              # чем больше число, тем лучше. От 0 до 1
      "\nMSE: ", mean_squared_error(y_test, y_pred)) # чем меньше, тем лучше. от 0 до бесконечности

r2:  0.7402240380163283 
MSE:  3.7882211538461563


<ipython-input-252-6d0e2b802b15>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_value, inplace=True)


Итог таков, что случайные деревья(RandomForestRegressor) показали себя лучше всех. Есть ещё пара моделей и классификаторов, которые можно проверить + в knn моделе можно изменять кол-во соседей, это тоже влияет на результат.

Для замены пропусков в одном столбце, я временно заменял в других столбцах NaNы медианой значений стобца. Возможно можно использовав столбец DateTime найти закономерность и сделать алгоритм, который будет подставлять более точные значения. Постараюсь этим заняться.

Если убрать столбец DateTime, то df.duplicated().any

# Алгоритм для замены NaNов - за этим алгоритмом вы сюда пришли.

В чём суть:
Мы берём все столбцы кроме **'DateTime'** и

1.   Мы берём все столбцы кроме **'DateTime'** и **'stage_4_output_danger_gas'**(почему? Объясню в конце текста).
2.   Разделяем датафрейм на Столбец, в котором необходимо заполнить пропуски, и датафрейм с остальными столбцами.
3.   С помощью отечественной функции super_train_split() разделяем данные на Данные для обучения(X_train, y_train) и Данные для предсказания(X_test). y_temp - датафрейм NaNов, он нам не нужен.
4.   Обучаем модель с помощью тренировочных данных
5.   Предсказываем значения, которые должны стоять вместо NaNов
6. Вставляем предсказанные данные в датафрейм.

Зачем я удалил **'DateTime'** и **'stage_4_output_danger_gas'**?

тип данных в **'DateTime'** не подходит для обработки с помощью регрессии(модель ругается на тип этого столбца). Так же DateTime не имеет пропусков и не является значимым для обучения.


**'stage_4_output_danger_gas'** имеет СЛИШКОМ много пропусков(пропусков около 3000 из 4400 строк). Если мы будем учитывать этот столб данных, то получим более неточный результат.

**Что делать с этим вам?**
Копируете библиотеки библиотеки, функцию super_train_split(), инициализацию регрессионной модели и этот алгоритм. После замены всех NaNов можно вставить удалённые столбцы обратно

*данные из DateTime вроде как можно поставить в качестве индекса

*Если заполнять столб **'stage_4_output_danger_gas'**, то заполнять его надо в самом конце, после всех итераций цикла(а это надо сделать, но у меня сейчас разрядится ноут, а мне ещё шесть уроков сидеть. 28.11.2024)

In [ ]:
df.pop('stage_4_output_danger_gas') # удаляем мешающий столбец

for column in df.columns: # для каждого столбца в df
  X = df.drop(column, axis=1) # все данные, кроме столбца column ()
  y = df[column] # наш столбец.

  X_train, X_test, y_train, y_temp = super_train_split(X, y) # Делим наши данные на
                                                             # на обучающую и тестовую выборку
  random_forest_model.fit(X_train, y_train) # обучаем модель(для каждого столбца)

  y_pred = random_forest_model.predict(X_test) # предсказываем пропущенные значения

  df.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений

"""
df.loc[X_test.index, column] = y_pred - как это работает?
X_test сохраняет у себя индексы строк с пропущенными данными
Главного датафрейма( в котором мы заменяем пропущенные значения).
Таким образом из y_pred списка значения подставляеются в нужные строки.
"""

['stage_1_output_konv_avd', 'stage_2_input_water_sum', 'stage_2_output_bottom_pressure', 'stage_2_output_bottom_temp', 'stage_2_output_bottom_temp_hum_steam', 'stage_2_output_bottom_vacuum', 'stage_2_output_top_pressure', 'stage_2_output_top_pressure_at_end', 'stage_2_output_top_temp', 'stage_2_output_top_vacuum', 'stage_3_input_pressure', 'stage_3_input_soft_water', 'stage_3_input_steam', 'stage_3_output_temp_hum_steam', 'stage_3_output_temp_top', 'stage_4_input_overheated_steam', 'stage_4_input_polymer', 'stage_4_input_steam', 'stage_4_input_water', 'stage_4_output_dry_residue_avg', 'stage_4_output_product', 'work_shift']


<ipython-input-30-4baee9d767cf>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(median_value, inplace=True)


x:  (241, 21)  / df:  (4400, 22)


<ipython-input-30-4baee9d767cf>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(median_value, inplace=True)


x:  (223, 21)  / df:  (4400, 22)


...

<ipython-input-30-4baee9d767cf>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(median_value, inplace=True)


x:  (182, 21)  / df:  (4400, 22)


<ipython-input-30-4baee9d767cf>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(median_value, inplace=True)


x:  (174, 21)  / df:  (4400, 22)


<ipython-input-30-4baee9d767cf>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(median_value, inplace=True)


Да, эта хрень ругается, что я не правильно использую метод **df[col].method(value, inplace=True)** в функции super_train_split. Но когда я заменяю мой код на предложенный консолью или интернетом, то у меня ничего не происходит. Испробовал все варианты, которые мог придумать

**ЕЩЁ ОДИН ВАЖНЫЙ МОМЕНТ**

последний столбец про смены имеет бинарный тип(или номинальный, не углублялся в этот момент). Алгоритм тоже его обрабатывает и заполняет "Не бинарными значениями"(то есть 1.235; 1.864 и тд). Это можно будет легко обратно превратить в бинарный тип данных, но возможно есть идеи лучше, рад буду выслушать.

**И вообще, если вам что-то не понравится, обязательно говорите!**

# По поводу дупликатов

Убирая столб DateTime, обнаруживается 159 строк, где значения всех столбцов = NaN. Их можно удалить до алгоритма, можно после

In [28]:
duplicates = df[df.duplicated(keep=False)]
duplicates # датафрейм с дубликатами

,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,stage_2_output_top_vacuum,...,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_danger_gas,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
1079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
duplicates = df.duplicated()

len(df[duplicates]) # кол-во дупликатов

159